In [1]:
# !pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import json
import os
import shutil

from bs4 import BeautifulSoup

In [11]:
driver = webdriver.Chrome('.\driver\chromedriver.exe')

In [14]:
perfumes = []
informations = []
notes = []
# reviews = [] 

# for num in range(26120000, 26160654 + 1):
for num in range(26120000, 26120010 + 1):
    driver.get(f"http://www.basenotes.net/ID{num}.html")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    
    ##### 이름, 연도, 이미지
    find_div = soup.find("h1", class_="fragranceheading")
    try:
        perfume_subhead = find_div.find("span", "subhead")
    except: 
        continue
    temp = dict()
    temp["id"] = num 
    temp["name"] = find_div.find("span").get_text()
    temp["year"] = perfume_subhead.find("a").get_text()
    temp["src"] = perfume_subhead.find("img")["src"]
    perfumes.append(temp)

    
    ##### 성별, 생산, 회사
    find_div = soup.find("div", class_ = "dirwrap50 frontdiv")
    table = find_div.find('table')
    df = pd.read_html(str(table))
    temp = dict()
    for d in df[0].values[1:]:
        if d[0] in "Gender":
            temp["gender"] = d[2]
        if d[0] in "Availability":
            temp["availability"] = d[2]
        if d[0] in "House":
            temp["house"] = d[2]
    temp["perfume_id"] = num
    informations.append(temp)

    
    ###### 향수 성분
    find_div = soup.find("div", class_="dirright50")
    find_ul = find_div.find_all("ul")
    temp = dict()
    for idx, link in enumerate(find_ul):
        for a_tag in link.find_all("a"):
            if idx == 0:
                temp["top"] = a_tag.get_text()
            elif idx == 1:
                temp["heart"] = a_tag.get_text()
            elif idx == 2:
                temp["base"] = a_tag.get_text()
    temp["perfume_id"] = num
    notes.append(temp)    
    
    
    ####### 리뷰
#     find_good_reviews = soup.find_all("div", class_ = "reviewmain review3")
#     find_neutral_reviews = soup.find_all("div", class_ = "reviewmain review2")
#     find_bad_reviews = soup.find_all("div", class_ = "reviewmain review1")
#     for idx, good_reviews in enumerate(find_good_reviews):
#         for review in good_reviews.find_all("div", class_="reviewblurb"):
#             text = review.get_text()
#             reviews.append({'review': text, 'react': 'good', 'perfume_id': num})
#     for idx, neutral_reviews in enumerate(find_neutral_reviews):
#         for review in neutral_reviews.find_all("div", class_="reviewblurb"):
#             text = review.get_text()
#             reviews.append({'review': text, 'react': 'neutral', 'perfume_id': num})
#     for idx, bad_reviews in enumerate(find_bad_reviews):
#         for review in bad_reviews.find_all("div", class_="reviewblurb"):
#             text = review.get_text()
#             reviews.append({'review': text, 'react': 'bad', 'perfume_id': num})

# data = {"perfumes": perfumes, "informations": informations, 'notes': notes, "reviews": reviews }

data = {"perfumes": perfumes, "informations": informations, 'notes': notes }

with open('./data/test2.json', 'w', encoding="utf-8") as make_file:
    json.dump(data, make_file, ensure_ascii=False, indent="\t")